# **DistillBERT Model**







> importo librerie



In [20]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from collections import defaultdict
import time



> Lettura DataSet e trasformazione dei dati in formato String per essere poi suddivisi in maniera randomica per l'adddestramento del modello



In [2]:
#caricamento del dataset
data = pd.read_csv("data.csv")
#creo nuova colonna nel dataset dove metto 1 se nella colonna 3 compare attack altrimenti metto 0
data["label"] = np.where(data["class3"] == "Attack", 1, 0)

#creo testo per addestare modello distillbert
#converto ogni colonna nel formato string e se nella cella della colonna del dataset risulta
#senza dato verrà sostituito con stringa vuota
scr = data["Scr_IP"].astype(str).replace("nan", "")
des = data["Des_IP"].astype(str).replace("nan", "")
proto = data["Protocol"].astype(str).replace("nan", "")
serv = data["Service"].astype(str).replace("nan", "")
alert = data["OSSEC_alert"].astype(str).replace("nan", "")

#creo nel dataset colonna nella quale metto sottoforma di string il contenuto della riga
data["text"] = pd.Series(
    np.where(scr != "", "Da " + scr + " ", "") +
    np.where(des != "", "A " + des + " ", "") +
    np.where(proto != "", "Protocollo " + proto + " ", "") +
    np.where(serv != "", "Servizio " + serv + " ", "") +
    np.where((alert != "") & (alert != "0"), "Alert " + alert, "")
).str.strip()

#divido il set di dati per addestrare il modello (60% dei dati) e sia per testarlo(40%)
train_df, test_df = train_test_split(
    data[["text", "label"]],
    test_size=0.4,
    stratify=data["label"],
    random_state=42
)
data = pd.read_csv("data.csv")



> Tokenizzazzione dati sia per l'addestramento e sia per la valutazione del modello






In [3]:
#tokenizzo sia i dati di addestramento che quelli di test
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_enc = tokenizer(list(train_df["text"]), padding=True, truncation=True, max_length=128, return_tensors="pt")
test_enc  = tokenizer(list(test_df["text"]),  padding=True, truncation=True, max_length=128, return_tensors="pt")

# Aggiungo le etichette
train_enc['labels'] = torch.tensor(train_df["label"].values)
test_enc['labels']  = torch.tensor(test_df["label"].values)

train_dataset = TensorDataset(train_enc['input_ids'], train_enc['attention_mask'], train_enc['labels'])
test_dataset  = TensorDataset(test_enc['input_ids'],  test_enc['attention_mask'],  test_enc['labels'])

#creazione dataset e dataloder
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, pin_memory=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(




> Selezione disposito GPU o CPU e creazione modello DistillBERT



In [4]:
#seleziono se utilizzare la gpu se è libera oppure presente nel dispositivo altrimenti utilizza la gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

#scarico modello pre-addestrato di ditillbert ingrado di fare delle classificazioni
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2 #utilizzo lables = 2 perchè devo fare due tipi di classificazione (attacco e normale)
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5) #ottimizzo i parametri e passo la velocità con cui il modello impara


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




> Addestramento del modello



In [5]:
#Addestro il modello
model.train()
epoche = 3 #scelto 3 perchè è così ho prestazioni superiori al 90% anche
for ep in range(epoche): #il modello scansiona i dati per essere addestrato x il numero di epoche
    tot_loss = 0
    for input_ids, attention_mask, labels in train_loader: #prendo i dati per l'allenamento
        input_ids = input_ids.to(device, non_blocking=True) #sposto il testo tokenizzato sulla gpu per rendere più veloce l'esecuzione
        attention_mask = attention_mask.to(device, non_blocking=True) #spsoto nella gpu sullo le parti di testo che non sono rimpieti con il padding
        labels = labels.to(device, non_blocking=True) #porto sulla gpu i valori per indicare se si tratta di un attacco o meno
        optimizer.zero_grad() #azzero i parametri così nella prossima epoca i valori non si sommino
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss #il modello legge il testo e cerca di capire se si tratta di un attacco o meno
        #infine calcola quanto ha sbagliato
        loss.backward() #calcola di quanto devono essere aggiornati i pesi dei parametri per la prossima epoca
        optimizer.step()#cambio i pesi dei parametri (il modello ha "imparato")

        tot_loss += loss.item() #calcolo la media dell'errore alla fine dell'epoca
    print(f"Epoca {ep+1}: Loss media = {tot_loss/len(train_loader):.4f}")


Epoca 1: Loss media = 0.5676
Epoca 2: Loss media = 0.2991
Epoca 3: Loss media = 0.1602




> Valutazione del modello



In [6]:
#valuto il modello
model.eval() #setto il modello in mdalità valutazione così che non aggiorni più i pesi dei parametri
all_preds = []
all_labels = []

with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:#prendo i dati per effettuare il test del modello
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits #il modello effettua la previsione
        preds = logits.argmax(dim=1) #sceglie se si tratta di un attacco oppure no

        all_preds.extend(preds.cpu().numpy()) #salva le predizioni e le sposta sulla cpu in formato numpy
        all_labels.extend(labels.cpu().numpy())#salva i dati del batch e li sposta sulla cpu in formato numpy

# Calcolo metriche
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"\nAccuracy:  {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall:    {recall*100:.2f}%")
print(f"F1-score:  {f1*100:.2f}%")


Accuracy:  92.25%
Precision: 88.79%
Recall:    96.45%
F1-score:  92.46%


#  **AI Multi Agent**



> Agente che si occupa di leggere i pacchetti di rete. Questo agente non fa alcuna elaborazione: si limita a “passare avanti” i log al prossimo agente nella catena. (Utile per sviluppi futuri come il filtraggio di pacchetti,normalizzarli o rimuovere dati sensibili prima di passarli agli altri agenti.)



In [7]:
class LogAgent:
  #metodo che legge i pacchetti di rete li restituisce in formato String all'agente successivo
    def observe(self, log_text: str) -> str:
        return log_text




> Agente responsabile di classificare i log come attacco o normale usando il modello DistilBERT.



In [8]:
class DetectionAgent:
  #costruttore della classe nella quale passo il modello DistillBERT addestrato precedentemente,
  #il tokenizer di DistillBert con il quale posso leggere i pacchetti di rete e trasformarli in numeri in modo tale che il modello possa leggerli,
  #il divice ovvero cpu o gpu in base a quale sia libero
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.eval()       #metto il modello in modalità valutazione in  modo tale da disattivare l'aggiornamento dei pesi

    #metodo che prende in input il pacchetto e restituisce 1 se si tratta di un attacco, 0 altrimenti
    def detect(self, text: str):
        inputs = self.tokenizer(
            text,
            return_tensors="pt",  #output in tensori pytorch
            truncation=True,      #tronca la stringa (paccetto trasformato in stringa) se troppo lungo
            max_length=128
        ).to(self.device)

        with torch.no_grad():                         #no_grad() disattiva il gradiente per mettere il modello DistillBert in modalità valutazine
            logits = self.model(**inputs).logits      #il modello classifica il pacchetto come attacco(1) oppure come normale(0)
            probs = torch.softmax(logits, dim=1)      #converte i logits in probabilità tra 0 e 1 per ciascuna classe.

        label = probs.argmax().item()                 #prende la classe con probabilità maggiore
        confidence = float(probs[0][label])           #salva la probabilità associata alla classe scelta.
        return label, confidence                      #restituisce una tupla contentente la classe con probabilità maggiore e la probabilità associata alla classe



> Agente ceh rileva attacchi basati su pattern statistici come il ricevimento di molti pacchetti in pochi secondi da uno stesso indirizzzo Ip sorgente



In [9]:
class StatisticalAgent:
  #costruttore della classe che inizializza la soglia massima dei pacchetti ricevuti in pochi secondi dallo stesso ip prima
  #che essi vrengono classificati come sospetti
    def __init__(self, rate_threshold=10):
        self.rate_threshold = rate_threshold
        self.ip_counter = defaultdict(list)   #creo una lista vuota se l'ip sorgente non esiste ancora e memorizzo il time stamp

    #metodo che analizza il pachetto e decide se esso sia sospetto o meno
    def analyze(self, text: str):
        now = time.time()         #salvo timestamp in secondi

        #estraggo indirizzo ip sorgente dal pachetto
        if "Da " in text:
            src_ip = text.split("Da ")[1].split(" ")[0]
            self.ip_counter[src_ip].append(now)       #aggiungo timestamp alla lista

            self.ip_counter[src_ip] = [t for t in self.ip_counter[src_ip] if now - t <= 10]       #mantengo i pacchetti degli ultime 10 secondi

            #se il numero di paccehtti ricevuti negli ultimi 10 secondi è maggiore della soglia setta nel costruttore il paccehtto viene consederato come attacco
            if len(self.ip_counter[src_ip]) >= self.rate_threshold:
                return 1, 0.9  #attacco con alta confidenza

        return 0, 0.0   #il pachetto non è consiederato un attacco



> Agente tiene traccia della storia dei pacchetti per ogni IP e usa questa memoria per aiutare nella decisione se un IP è sospetto.


> **Osservazione**: Il ContextAgent così com’è funziona bene nei test controllati, ma in situazioni reali di rete NAT/pubblica potrebbe generare falsi positivi perché si basa solo sull’IP.
Per ridurre questo rischio, bisognerebbe aggiungere altre feature oltre all’IP per identificare un utente o sessione in modo più preciso.





In [10]:
class ContextAgent:
    #costruttore della classe ceh crea una lista che associa a ogni indirizzo ip sorgente un contattore di eventi sospetti
    def __init__(self):
        self.history = defaultdict(int)   #creo lista se l'Ip non e presente

    #aggiorno la memoria ogni volta ceh viene processato un pacchetto
    def update(self, text: str, label: int):
      #estarggo indirizzo ip sorgente
        if "Da " in text:
            src_ip = text.split("Da ")[1].split(" ")[0]
            #se il pacchetto è stato classificato come attacco dal DetectionAgent incremento di 1 il contatore
            if label == 1:
                self.history[src_ip] += 1

    #funzione che valuta il pacchetto in base alla memoria dell'agente
    def assess(self, text: str):
        #estarggo indirizzo ip sorgente
        if "Da " in text:
            src_ip = text.split("Da ")[1].split(" ")[0]
            #se l'Ip ha almeno 3 eventi sospetti lo considero attacco
            if self.history[src_ip] >= 3:
                return 1, 0.8
        return 0, 0.0   #classifico il pacchetto come normale



> Agente che combina le predizioni svolte dagl'altri agenti e seleziona il risultato migliore



>**Osservazione:** L'agente prende solo in cosiderazione la confidenza più alta che ha stabilito uno degli agenti. Sarebbe più efficace se si facesse una media pesata di tali confidenze







In [11]:
class FusionAgent:
    #Funzione che prende in input tre tuple
    #ml_out tupla del agentDetection
    #stat_out tupla del StatisticalAgent
    #ctx_out tupla del ContextAgent
    def fuse(self, ml_out, stat_out, ctx_out):
        votes = [ml_out, stat_out, ctx_out]       #metto i voti in una lista
        #prendi il voto con la confidenza massima
        label = max(votes, key=lambda x: x[1])[0]     #prendo la incosiderazione solo la tupla che contiene il livello di confidenza più alto
        confidence = max(v[1] for v in votes)         #prendo il valore massimo di confidenza
        return label, confidence



> Agente che trasforma una decisione in un'azione di sicurezza



In [12]:
class PolicyAgent:
    #costruttore che stabilisce la soglia minima di confidenza prima di bloccare il pacchetto classificato come attacco
    def __init__(self, threshold=0.85):     #threshold=0.85 perchè valore di default di python
        self.threshold = threshold

    #funzione ceh prende in input il risultato del FusionAgent
    def decide(self, label, confidence):
        #decide cosa fare con il paccehtto
        if label == 1 and confidence >= self.threshold:   #Contralla se il pacchetto è stato classificato come attacco e la confidenzialità del pacchetto
            return "BLOCK"                                #paccehtto viene bloccatto in caso sia classificato come attacco e la confidenza è maggiore o uguale 85%
        elif label == 1:
            return "ALERT"                                #viene inviato un alert in caso sia classificato come attacco e la confidenza è minore 85%
        else:
            return "ALLOW"                                #il paccehtto viene lasciato passare



> Agente che esegue le decisioni del PolicyAgent


> **Osservazione**: questo agente risulta momentaneamente inutile. In questo caso è usato solo per livello strutturale per separare decisione da esecuzione. Utile in futuro per applicare meccanismi di sicurezza in base all'esito della decisione.





In [13]:
class ResponseAgent:
    #restituisce semplicemente la risposta del PolicyAgent
    def act(self, action, confidence):
        return action, confidence



> Agente che modifica il comportamento del sistema nel tempo



> **Osservazione**: tale agente non modifica il modello DistilBERT. Osserva gli errori di classificazione dei pacchetti e modifica il comportamento futuro del sistema. Non modifica le decisoni del PolicyAgent ma rende adattava la sua politica di decisione. In futuro potrebbe modificare la decisione del policyAgent per rendere il sistema più performante. Potrebbe anceh estendere la funzionalità di ri-addestrare il modello DistillBERT





In [14]:
class LearningAgent:
    #costruttore che prende in input la decisione del PolicyAgent
    def __init__(self, policy_agent):
        self.policy_agent = policy_agent
        self.feedback_log = []  #salva (pred, reale, conf) per costruire memoria degli errori

    #funzione che viene chiamata ogni volta che viene presa una decisione
    def record_feedback(self, predicted_label, true_label, confidence):
        self.feedback_log.append((predicted_label, true_label, confidence))
        self.adjust_threshold()       #serve per addattare il sistema

    def adjust_threshold(self):
        if not self.feedback_log:
            return

        fp = 0  #falsi positivi ovvero il sistema classifica il pacchetto come attacco quando non lo è
        fn = 0  #falsi negativi ovvero il sistema classifica il paccehtto come normale quanodo è un attacco

        #considero ultimi 50 log
        #stimo di quanto il sistema baglia in una determinata direzione
        for pred, true, _ in self.feedback_log[-50:]:
            if pred == 1 and true == 0:
                fp += 1
            elif pred == 0 and true == 1:
                fn += 1

        #regolo soglia
        if fn > fp and self.policy_agent.threshold > 0.6:       #se il sistema lascia passare troppi falsi-negativi regolo la soglia
            self.policy_agent.threshold -= 0.02                 #abbasso la soglia
        elif fp > fn and self.policy_agent.threshold < 0.95:    #se il sistema lascia passare troppi falsi-positivi regolo la soglia
            self.policy_agent.threshold += 0.02                 #abbasso la soglia

        self.policy_agent.threshold = max(0.5, min(0.95, self.policy_agent.threshold))   #regolo livello di confidenzialità nel policyAgent




> Classe che cordina tutti gli agenti. Definisce l'ordine corretto di esecuzione. Riceve pachetti/o, lo fa analizzare da più agenti, fonde i risultati e produce una risposta.



In [15]:
class IntrusionDetectionSystem:
    #costruttore che prende in input gli agenti già istanziati per avere acesso a tutte le parti del sistema
    def __init__(self,
                 log_agent,
                 detection_agent,
                 statistical_agent,
                 context_agent,
                 fusion_agent,
                 policy_agent,
                 response_agent):
        self.log_agent = log_agent
        self.detection_agent = detection_agent
        self.statistical_agent = statistical_agent
        self.context_agent = context_agent
        self.fusion_agent = fusion_agent
        self.policy_agent = policy_agent
        self.response_agent = response_agent

    #funzione ceh prende in input il/i pacchetti
    def process_log(self, log_text):
        text = self.log_agent.observe(log_text)   #passo i pacchetti al LogAgent che gli/lo osserva
        #faccio analizzare il pacchetto agli agenti competenti
        ml_label, ml_conf = self.detection_agent.detect(text)         #classifica se il/i pacchetto/i è/sono un attacco o sono normali
        stat_label, stat_conf = self.statistical_agent.analyze(text)  #analizzo ogni quanti secondi ricevo tale pachetto/i e stabilisco se sono attacco/hi
        ctx_label, ctx_conf = self.context_agent.assess(text)         #analizzo la storia del pacchetto e classifico se si tratta di un attacco o meno
        final_label, final_conf = self.fusion_agent.fuse(             #combino le tre analisi e ridico gli errori dei singoli agenti
            (ml_label, ml_conf),
            (stat_label, stat_conf),
            (ctx_label, ctx_conf)
        )

        self.context_agent.update(text, final_label)    #il PolicyAgent sceglie cosa fare in ase alla decisone del FusionAgent

        action = self.policy_agent.decide(final_label, final_conf)
        return action, final_conf, ml_label  #ritorno anche label ML per feedback

In [23]:
policy_agent = PolicyAgent()  #creo istanza PolicyAgent
learning_agent = LearningAgent(policy_agent)    #creo istanza LearingAgent
#creo istanza IntrusionDetectionSystem
ids = IntrusionDetectionSystem(
    LogAgent(),
    DetectionAgent(model, tokenizer, device), #passo il modello DistillBERT, il tokenizer di pytorch e il device
    StatisticalAgent(rate_threshold=5),
    ContextAgent(),
    FusionAgent(),
    policy_agent,
    ResponseAgent()
)

#esempio di log
logs = [
    ("Da 192.168.1.10 A 10.0.0.1 Protocollo udp Servizio dns", 0),
    ("Da 192.168.1.10 A 10.0.0.1 Protocollo udp Servizio dns", 1),
    ("Da 192.168.1.10 A 10.0.0.1 Protocollo udp Servizio dns", 1),
]

for log_text, true_label in logs:
    action, conf, ml_label = ids.process_log(log_text)
    print(f"Azione: {action}, Confidenza: {conf:.2f}, Soglia attuale: {policy_agent.threshold:.2f}")
    #aggiorno il LearningAgent con il feedback reale
    learning_agent.record_feedback(ml_label, true_label, conf)

Azione: ALLOW, Confidenza: 0.97, Soglia attuale: 0.85
Azione: ALLOW, Confidenza: 0.97, Soglia attuale: 0.85
Azione: ALLOW, Confidenza: 0.97, Soglia attuale: 0.83
